# Import all modules and load data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os


pd.set_option('display.max_columns', 90)

In [2]:
loan_data = pd.read_csv('accepted_2007_to_2018Q4_Copy.csv')
loan_data.shape

/Users/jjobes/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(29999, 151)

# Remove columns with no data

Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types

In [3]:
# find missing values for each column


missing = pd.concat([loan_data.isnull().sum(), 100 * loan_data.isnull().mean()], axis=1)
missing.columns=['count', '%']
missing.sort_values(by='count')


,count,%
id,0,0.0
pct_tl_nvr_dlq,0,0.0
tot_cur_bal,0,0.0
tot_coll_amt,0,0.0
acc_now_delinq,0,0.0
...,...,...
sec_app_open_acc,29999,100.0
sec_app_mort_acc,29999,100.0
sec_app_inq_last_6mths,29999,100.0
sec_app_fico_range_high,29999,100.0


In [4]:
# define list of columns with a large amount of missing values

large_null_columns = missing[missing['%'] > 0.99] 
large_null_columns.reset_index(inplace=True)
large_null_columns.columns

columns_to_drop = list(large_null_columns['index'])
print(columns_to_drop)

['member_id', 'emp_title', 'emp_length', 'desc', 'mths_since_last_delinq', 'mths_since_last_record', 'next_pymnt_d', 'mths_since_last_major_derog', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'bc_util', 'mo_sin_old_il_acct', 'mths_since_recent_bc_dlq', 'mths_since_recent_inq', 'mths_since_recent_revol_delinq', 'num_tl_120dpd_2m', 'percent_bc_gt_75', 'revol_bal_joint', 'sec_app_fico_range_low', 'sec_app_fico_range_high', 'sec_app_earliest_cr_line', 'sec_app_inq_last_6mths', 'sec_app_mort_acc', 'sec_app_open_acc', 'sec_app_revol_util', 'sec_app_open_act_il', 'sec_app_num_rev_accts', 'sec_app_chargeoff_within_12_mths', 'sec_app_collections_12_mths_ex_med', 'sec_app_mths_since_last_major_derog', 'hardship_type', 'hardship_reason', 'hardship_status', 'deferral_term', 'ha

In [5]:
# drop these columns with no data

new_data = loan_data.drop(columns_to_drop, axis=1)
new_data.shape

(29999, 86)

# Reset Index to loan id

In [6]:
# set index to something useful

new_data.set_index('id', inplace=True)

# Split dataframe into numeric and non-numeric

Split data into numeric and non_numeric valued dataframes

In [10]:
numeric_df = new_data.select_dtypes(np.number)

numeric_columns = list(numeric_df)


non_numeric_df = new_data.drop(numeric_columns, axis=1)

non_numeric_df_columns = list(non_numeric_df)

In [11]:
numeric_df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,policy_code,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
68407277,3600,3600,3600,13.99,123.03,55000.0,5.91,0,675,679,1,7,0,2765,29.7,13,0.00,0.00,4421.723917,4421.72,3600.00,821.72,0.0,0.0,0.0,122.67,564,560,0,1,0,722,144904,9300,4,20701,1506.0,0,0,128,3,3,1,4.0,2,2,4,2,5,3,4,9,4,7,0,0,3,76.9,0,0,178050,7746,2400,13734
68355089,24700,24700,24700,11.99,820.28,65000.0,16.06,1,715,719,4,22,0,21470,19.2,38,0.00,0.00,25679.660000,25679.66,24700.00,979.66,0.0,0.0,0.0,926.35,699,695,0,1,0,0,204396,111800,4,9733,57830.0,0,0,192,2,2,4,2.0,0,5,5,13,17,6,20,27,5,22,0,0,2,97.4,0,0,314017,39475,79300,24667
68341763,20000,20000,20000,10.78,432.66,63000.0,10.78,0,695,699,0,6,0,7869,56.2,18,0.00,0.00,22705.924290,22705.92,20000.00,2705.92,0.0,0.0,0.0,15813.30,704,700,0,1,0,0,189699,14000,6,31617,2737.0,0,0,184,14,14,5,101.0,0,2,3,2,4,6,4,7,3,6,0,0,0,100.0,0,0,218418,18696,6200,14877
66310712,35000,35000,35000,14.85,829.90,110000.0,17.06,0,785,789,0,13,0,7802,11.6,17,15897.65,15897.65,31464.010000,31464.01,19102.35,12361.66,0.0,0.0,0.0,829.90,679,675,0,1,0,0,301500,67300,2,23192,54962.0,0,0,87,2,2,1,2.0,0,4,5,8,10,2,10,13,5,13,0,0,1,100.0,0,0,381215,52226,62500,18000
68476807,10400,10400,10400,22.45,289.91,104433.0,25.37,1,695,699,3,12,0,21929,64.5,35,0.00,0.00,11740.500000,11740.50,10400.00,1340.50,0.0,0.0,0.0,10128.96,704,700,0,1,0,0,331730,34000,10,27644,4567.0,0,0,210,4,4,6,4.0,0,4,6,5,9,10,7,19,6,12,0,0,4,96.6,0,0,439570,95768,20300,88097


In [12]:
non_numeric_df.head()

,term,grade,sub_grade,home_ownership,verification_status,issue_d,loan_status,pymnt_plan,url,purpose,title,zip_code,addr_state,earliest_cr_line,initial_list_status,last_pymnt_d,last_credit_pull_d,application_type,hardship_flag,disbursement_method,debt_settlement_flag
id,,,,,,,,,,,,,,,,,,,,,
68407277,36 months,C,C4,MORTGAGE,Not Verified,15-Dec,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,Debt consolidation,190xx,PA,3-Aug,w,19-Jan,19-Mar,Individual,N,Cash,N
68355089,36 months,C,C1,MORTGAGE,Not Verified,15-Dec,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,small_business,Business,577xx,SD,Dec-99,w,16-Jun,19-Mar,Individual,N,Cash,N
68341763,60 months,B,B4,MORTGAGE,Not Verified,15-Dec,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,home_improvement,NaN,605xx,IL,Aug-00,w,17-Jun,19-Mar,Joint App,N,Cash,N
66310712,60 months,C,C5,MORTGAGE,Source Verified,15-Dec,Current,n,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,Debt consolidation,076xx,NJ,8-Sep,w,19-Feb,19-Mar,Individual,N,Cash,N
68476807,60 months,F,F1,MORTGAGE,Source Verified,15-Dec,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,major_purchase,Major purchase,174xx,PA,Jun-98,w,16-Jul,18-Mar,Individual,N,Cash,N


In [15]:
numeric_df.dtypes.value_counts()
numeric_df.shape

(29999, 64)

In [17]:
non_numeric_df.dtypes.value_counts()
non_numeric_df.shape

(29999, 21)

# Review numeric dataframe

In [18]:
numeric_df.describe()

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,policy_code,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,29999.000000,29999.000000,29999.000000,29999.000000,29999.000000,2.999900e+04,29998.000000,29999.000000,29999.000000,29999.000000,29999.000000,29999.000000,29999.000000,29999.000000,29984.000000,29999.000000,29999.000000,29999.000000,29999.000000,29999.000000,29999.000000,29999.000000,2.999900e+04,29999.000000,29999.000000,29999.000000,29999.000000,29999.000000,29999.000000,29999.0,29999.000000,29999.000000,2.999900e+04,29999.000000,29999.000000,29999.000000,29714.000000,29999.000000,29999.000000,29999.000000,29999.000000,29999.000000,29999.000000,29723.000000,29999.000000,29999.000000,29999.000000,29999.000000,29999.000000,29999.000000,29999.000000,29999.000000,29999.000000,29999.000000,29999.000000,29999.000000,29999.000000,29999.000000,29999.000000,29999.000000,2.999900e+04,29999.000000,29999.000000,29999.000000
mean,15000.318344,15000.318344,14992.248908,12.242534,434.339515,7.888676e+04,19.408835,0.337011,694.866329,698.866496,0.604353,12.101137,0.240075,17378.464715,51.690928,25.461315,1000.696043,1000.072461,15001.064742,14993.273176,12016.414316,2750.998200,2.179053e+00,231.473155,41.286640,4216.472136,677.288810,661.656055,0.023334,1.0,0.004900,274.673056,1.429451e+05,34692.337378,4.855595,13252.029901,10502.955442,0.008734,11.587086,185.777059,13.211507,7.553185,1.645555,23.991757,0.538451,3.721724,5.680256,4.825994,7.981866,8.871896,8.462315,14.667722,5.731624,12.052235,0.003300,0.089436,2.255475,93.925818,0.134738,0.068802,1.780269e+05,53835.934464,22662.980033,45894.863829
std,8709.659995,8709.659995,8704.031994,4.209396,248.658024,8.537145e+04,10.507178,0.891299,31.194369,31.195174,0.888344,5.750331,0.645234,22564.015374,24.241345,12.134515,3012.649685,3010.641706,9766.227565,9760.240853,8367.505518,2772.138531,1.450583e+01,974.852343,174.993972,6441.603183,79.384708,126.778695,0.165300,0.0,0.071715,2181.606994,1.566171e+05,34182.537226,3.303595,15880.610380,15678.814831,0.098950,713.976242,96.541710,17.066413,8.855886,1.928312,31.614545,1.373040,2.331207,3.360008,3.072364,4.796534,7.582831,4.776675,8.281632,3.340622,5.733512,0.058504,0.490494,1.909322,8.901241,0.382870,0.439332,1.748295e+05,49376.856963,22354.588352,44959.295929
min,1000.000000,1000.000000,950.000000,5.320000,30.540000,0.000000e+00,0.000000,0.000000,660.000000,664.000000,0.000000,1.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.000000e-10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,1.000000,0.000000,0.000000,0.000000,12.500000,0.000000,0.000000,2.500000e+03,0.000000,0.000000,0.000000
25%,8000.000000,8000.000000,8000.000000,9.170000,255.030000,4.800000e+04,12.700000,0.000000,670.000000,674.000000,0.000000,8.000000,0.000000,6204.000000,33.500000,17.000000,0.000000,0.000000,7345.478359,7341.745000,5600.000000,893.830000,0.000000e+

In [22]:
list(numeric_df.columns)

['loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'int_rate',
 'installment',
 'annual_inc',
 'dti',
 'delinq_2yrs',
 'fico_range_low',
 'fico_range_high',
 'inq_last_6mths',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_amnt',
 'last_fico_range_high',
 'last_fico_range_low',
 'collections_12_mths_ex_med',
 'policy_code',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'total_rev_hi_lim',
 'acc_open_past_24mths',
 'avg_cur_bal',
 'bc_open_to_buy',
 'chargeoff_within_12_mths',
 'delinq_amnt',
 'mo_sin_old_rev_tl_op',
 'mo_sin_rcnt_rev_tl_op',
 'mo_sin_rcnt_tl',
 'mort_acc',
 'mths_since_recent_bc',
 'num_accts_ever_120_pd',
 'num_actv_bc_tl',
 'num_actv_rev_tl',
 'num_bc_sats',
 'num_bc_tl',
 'num_il_tl',
 'num_op_rev_tl',
 'num_rev_accts',
 'num_rev_tl_bal_gt_0',
 'num_sats',


In [23]:
for i in list(numeric_df.columns):
    print(numeric_df[i].value_counts())

10000    2030
15000    1733
12000    1664
20000    1621
35000    1351
         ... 
32025       1
30550       1
26775       1
29525       1
31450       1
Name: loan_amnt, Length: 1206, dtype: int64
10000    2030
15000    1733
12000    1664
20000    1621
35000    1351
         ... 
32025       1
30550       1
26775       1
29525       1
31450       1
Name: funded_amnt, Length: 1206, dtype: int64
10000    1992
15000    1667
12000    1596
20000    1510
35000    1087
         ... 
33075       1
6175        1
2125        1
28850       1
32800       1
Name: funded_amnt_inv, Length: 1251, dtype: int64
11.99    2016
9.17     1733
10.64    1547
9.76     1488
11.22    1436
         ... 
26.06      13
26.99      10
28.99       9
27.49       7
27.99       5
Name: int_rate, Length: 64, dtype: int64
318.79    148
315.12    132
325.69    126
332.10    125
321.55    124
         ... 
476.60      1
773.18      1
424.50      1
154.08      1
31.00       1
Name: installment, Length: 8203, dtype: int64
600

In [24]:
# all loan amounts = funded amount. No need to have duplicate data so drop funded amount and the column. 
# funded_amount_inv doesnt always equal loan_amount so I will leave this column

#numeric_df['funded_amnt'].describe()

#numeric_df['diff_loan_funded'] = numeric_df['loan_amnt'] - numeric_df['funded_amnt']
#numeric_df['diff_loan_funded'].describe()

del numeric_df['funded_amnt']


In [25]:
numeric_df.shape

(29999, 63)

In [26]:
numeric_df.head()

,loan_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,policy_code,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
68407277,3600,3600,13.99,123.03,55000.0,5.91,0,675,679,1,7,0,2765,29.7,13,0.00,0.00,4421.723917,4421.72,3600.00,821.72,0.0,0.0,0.0,122.67,564,560,0,1,0,722,144904,9300,4,20701,1506.0,0,0,128,3,3,1,4.0,2,2,4,2,5,3,4,9,4,7,0,0,3,76.9,0,0,178050,7746,2400,13734
68355089,24700,24700,11.99,820.28,65000.0,16.06,1,715,719,4,22,0,21470,19.2,38,0.00,0.00,25679.660000,25679.66,24700.00,979.66,0.0,0.0,0.0,926.35,699,695,0,1,0,0,204396,111800,4,9733,57830.0,0,0,192,2,2,4,2.0,0,5,5,13,17,6,20,27,5,22,0,0,2,97.4,0,0,314017,39475,79300,24667
68341763,20000,20000,10.78,432.66,63000.0,10.78,0,695,699,0,6,0,7869,56.2,18,0.00,0.00,22705.924290,22705.92,20000.00,2705.92,0.0,0.0,0.0,15813.30,704,700,0,1,0,0,189699,14000,6,31617,2737.0,0,0,184,14,14,5,101.0,0,2,3,2,4,6,4,7,3,6,0,0,0,100.0,0,0,218418,18696,6200,14877
66310712,35000,35000,14.85,829.90,110000.0,17.06,0,785,789,0,13,0,7802,11.6,17,15897.65,15897.65,31464.010000,31464.01,19102.35,12361.66,0.0,0.0,0.0,829.90,679,675,0,1,0,0,301500,67300,2,23192,54962.0,0,0,87,2,2,1,2.0,0,4,5,8,10,2,10,13,5,13,0,0,1,100.0,0,0,381215,52226,62500,18000
68476807,10400,10400,22.45,289.91,104433.0,25.37,1,695,699,3,12,0,21929,64.5,35,0.00,0.00,11740.500000,11740.50,10400.00,1340.50,0.0,0.0,0.0,10128.96,704,700,0,1,0,0,331730,34000,10,27644,4567.0,0,0,210,4,4,6,4.0,0,4,6,5,9,10,7,19,6,12,0,0,4,96.6,0,0,439570,95768,20300,88097


# Review non-numeric dataframe

In [28]:
non_numeric_df.head()

,term,grade,sub_grade,home_ownership,verification_status,issue_d,loan_status,pymnt_plan,url,purpose,title,zip_code,addr_state,earliest_cr_line,initial_list_status,last_pymnt_d,last_credit_pull_d,application_type,hardship_flag,disbursement_method,debt_settlement_flag
id,,,,,,,,,,,,,,,,,,,,,
68407277,36 months,C,C4,MORTGAGE,Not Verified,15-Dec,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,Debt consolidation,190xx,PA,3-Aug,w,19-Jan,19-Mar,Individual,N,Cash,N
68355089,36 months,C,C1,MORTGAGE,Not Verified,15-Dec,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,small_business,Business,577xx,SD,Dec-99,w,16-Jun,19-Mar,Individual,N,Cash,N
68341763,60 months,B,B4,MORTGAGE,Not Verified,15-Dec,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,home_improvement,NaN,605xx,IL,Aug-00,w,17-Jun,19-Mar,Joint App,N,Cash,N
66310712,60 months,C,C5,MORTGAGE,Source Verified,15-Dec,Current,n,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,Debt consolidation,076xx,NJ,8-Sep,w,19-Feb,19-Mar,Individual,N,Cash,N
68476807,60 months,F,F1,MORTGAGE,Source Verified,15-Dec,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,major_purchase,Major purchase,174xx,PA,Jun-98,w,16-Jul,18-Mar,Individual,N,Cash,N


In [29]:
non_numeric_df.shape

(29999, 21)

In [30]:
list(non_numeric_df.columns)

['term',
 'grade',
 'sub_grade',
 'home_ownership',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'url',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'earliest_cr_line',
 'initial_list_status',
 'last_pymnt_d',
 'last_credit_pull_d',
 'application_type',
 'hardship_flag',
 'disbursement_method',
 'debt_settlement_flag']

In [31]:
for i in list(non_numeric_df.columns):
    print(non_numeric_df[i].value_counts())

 36 months    20662
 60 months     9337
Name: term, dtype: int64
B    9115
C    8672
A    5246
D    4143
E    2098
F     604
G     121
Name: grade, dtype: int64
C1    2018
B4    1986
B3    1930
B5    1853
C4    1810
C2    1776
C3    1749
B2    1734
B1    1612
A5    1475
C5    1319
A1    1196
D1    1150
A4    1006
D2     903
A2     807
D3     778
A3     762
D4     719
D5     593
E1     538
E2     507
E3     441
E4     338
E5     274
F1     195
F2     147
F3     124
F4      71
F5      67
G1      41
G2      37
G3      21
G4      13
G5       9
Name: sub_grade, dtype: int64
MORTGAGE    14659
RENT        11846
OWN          3493
ANY             1
Name: home_ownership, dtype: int64
Source Verified    12094
Verified            9189
Not Verified        8716
Name: verification_status, dtype: int64
15-Dec    29999
Name: issue_d, dtype: int64
Fully Paid            21085
Charged Off            5413
Current                3256
Late (31-120 days)      154
In Grace Period          72
Late (16-30 days) 

In [32]:
# looks like purpose and title are the same thing. Lets see if they are

print(non_numeric_df['purpose'].isnull().sum())
print(non_numeric_df['title'].isnull().sum())

# purpose looks like the main catagory with no missing values
# are any contradictory? Need to investigate

0
132


In [33]:
purpose_labels = list(non_numeric_df['purpose'].value_counts().index)

purpose_labels

['debt_consolidation',
 'credit_card',
 'home_improvement',
 'other',
 'major_purchase',
 'medical',
 'small_business',
 'car',
 'vacation',
 'moving',
 'house',
 'renewable_energy']

In [34]:
title_labels = list(non_numeric_df['title'].value_counts().index)

title_labels

['Debt consolidation',
 'Credit card refinancing',
 'Home improvement',
 'Other',
 'Major purchase',
 'Medical expenses',
 'Business',
 'Car financing',
 'Vacation',
 'Moving and relocation',
 'Home buying',
 'Green loan']

In [25]:
# review with Tony
# filter new_data into each purpose catorgy then right for loop to confirm purpose matches title? seems like there is a better way

#for i in len(new_data):
#    if new_data['purpose'][0] == 'debt_consolidation' and new_data['title'][0] == 'Debt consolidation':
#    return True
    